# RV-NPL analysis for simulated genes

In this notebook I simulate family data with genotypes generated conditional on affection status. And use RV-NPL to analyze the simulated families.

## Workflow process

Here I analyze one gene (MYO7A), as hard-coded in this notebook:

```
sos run NPL_Simulation.ipynb analyze
```

In [3]:
First I simulate family data using RarePedSim.

First I simulate family data using RarePedSim.


In [1]:
[global]
# Path to the ped file
parameter: ped_file = './data/100extend01.ped'
# Variant information for genes to analyze
parameter: sfs_file = './data/MYO7A.sfs'
# conf file contains the simulation specifications
parameter: conf_file = './data/ComplexPhenotype.conf'
parameter: out_dir = './data/simulation'

[analyze]
depends: executable('rarepedsim')
output: f'{out_dir}/MYO7A/rep1.vcf'
bash: expand = '${ }'
    rarepedsim generate -s ${sfs_file} -c ${conf_file} -p ${ped_file} --num_genes 1 --num_reps 1 -o ${out_dir} --vcf --debug -b -1

## Results

<Add Result> 